In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [1]:
path_to_project = 'C:/Users\damis\Google Drive\ETH\Master thesis\KalmanNet_VO/'

In [2]:
import sys
sys.path.insert(0, path_to_project+'Simulations\Pendulum')
sys.path.insert(1,'/content/drive/MyDrive/ETH/Master thesis/KalmanNet_VO/')
sys.path.insert(2,'/content/drive/MyDrive/ETH/Master thesis/KalmanNet_VO/saved_models/')


In [3]:
import torch
from parameters import NL_T, NL_T_test, NL_m1_0, NL_m2_0, NL_m, NL_n
from model import f, h
from EKF_test_visual import EKFTest
from Linear_sysmdl_visual import SystemModel
from Extended_sysmdl_visual import SystemModel as NL_SystemModel
from KalmanNet_sysmdl import SystemModel as NewArch_SystemModel
from Extended_data_visual import DataGen,DataLoader,DataLoader_GPU, getObs
from Extended_data_visual import N_E, N_CV, N_T, F, F_rotated, T, T_test, m1_0, m2_0, m, n,H_fully_connected, H_matrix_for_visual, b_for_visual
from visual_supplementary import y_size, check_changs
from Pipeline_KF_visual import Pipeline_KF
from KalmanNet_nn_LinearCase_OldArch_visual import KalmanNetNN
from KalmanNet_nn_OldArch_visual import KalmanNetNN as Extended_KalmanNetNN
from KalmanNet_nn_NewArch_visual import KalmanNetNN as KalmanNetNN_NewArch
from main_AE import Autoencoder, Encoder, test_encoder

Running on GPU
['C:/Users\\damis\\Google Drive\\ETH\\Master thesis\\KalmanNet_VO/Simulations\\Pendulum', 'Simulations/Pendulum/', 'C:/Users\\damis\\Google Drive\\ETH\\Master thesis\\KalmanNet_VO/Simulations\\Pendulum\\..', '/content/drive/MyDrive/ETH/Master thesis/KalmanNet_VO/', '/content/drive/MyDrive/ETH/Master thesis/KalmanNet_VO/saved_models/', 'C:\\Users\\damis\\Google Drive\\ETH\\Master thesis\\KalmanNet_VO', 'C:\\Users\\damis\\Google Drive\\ETH\\Master thesis\\KalmanNet_VO', 'C:\\Users\\damis\\.conda\\envs\\torch\\python39.zip', 'C:\\Users\\damis\\.conda\\envs\\torch\\DLLs', 'C:\\Users\\damis\\.conda\\envs\\torch\\lib', 'C:\\Users\\damis\\.conda\\envs\\torch', '', 'C:\\Users\\damis\\.conda\\envs\\torch\\lib\\site-packages', 'C:\\Users\\damis\\.conda\\envs\\torch\\lib\\site-packages\\win32', 'C:\\Users\\damis\\.conda\\envs\\torch\\lib\\site-packages\\win32\\lib', 'C:\\Users\\damis\\.conda\\envs\\torch\\lib\\site-packages\\Pythonwin']


In [4]:
if torch.cuda.is_available():
   dev = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
   torch.set_default_tensor_type('torch.cuda.FloatTensor')
   print("Running on the GPU")
else:
   dev = torch.device("cpu")
   print("Running on the CPU")

Running on the GPU


In [5]:
r2 = torch.tensor([1e-4])
q2 = torch.tensor([1e-6]) # can be tuned
r = torch.sqrt(r2)
q = torch.sqrt(q2)
sys_model = NL_SystemModel(f, q, h, r, NL_T, NL_T_test, NL_m, NL_n, None)
sys_model.InitSequence(NL_m1_0, NL_m2_0)
Q_mod = q * q * torch.eye(NL_m)
R_mod = r * r * torch.eye(NL_n)
sys_model_KNet = NewArch_SystemModel(f, Q_mod, h, R_mod, NL_T, NL_T_test)
sys_model_KNet.InitSequence(NL_m1_0, NL_m2_0)
model_AE_conv_trained = Encoder(1)
model_AE_conv_trained.load_state_dict(torch.load(path_to_project+'saved_models/Only_conv_encoder.pt'))
model_AE_conv_trained = model_AE_conv_trained.to(dev, non_blocking=True)

In [6]:
dataFolderName = path_to_project+ '/Simulations/Pendulum' + '/'
dataFileName = r'C:\Users\damis\Google Drive\ETH\Master thesis\KalmanNet_VO\Datasets\Pendulum\pendulum_trans_std_0.001_obs_std_1e-05_train_5000&30_val_100&30_test_100&40.pt'
#dataFileName = "y24x24_Ttrain30_NE1000_NCV100_NT100_Ttest40_pendulum_train_encoder.pt"
data_name = 'pendulum'
[train_input, train_target, cv_input, cv_target, test_input, test_target] = DataLoader_GPU(dataFileName)


In [ ]:
ta

In [14]:
m =test_target.reshape((test_target.shape[0]*test_target.shape[1], test_target.shape[2]))
#m1 = torch.unsqueeze(m, 1)
m = m[:, 0:1]
print(m.shape)
print(m)

#print(m1.shape)
#print(m1)

torch.Size([4000, 1])
tensor([[0.3192],
        [0.3077],
        [0.2742],
        ...,
        [0.2604],
        [0.6466],
        [0.9874]])


In [32]:
oo = torch.transpose(test_target, 1,2)
aa = torch.reshape(oo, (4000, 2))
print(aa)
print(test_target)

tensor([[ 0.3192,  0.1884],
        [-3.1416, -1.8661],
        [ 0.3077,  0.2444],
        ...,
        [-7.8031,  4.2000],
        [ 1.1766,  0.9874],
        [-9.1972,  3.0938]])
tensor([[[  0.3192,  -3.1416,   0.3077,  ...,  -1.5067,   0.1184,  -1.6330],
         [  0.1884,  -1.8661,   0.2444,  ...,  -1.9967,  -0.1368,  -1.7563]],

        [[  1.3078,  -3.1416,   1.2724,  ...,   2.7803,  -0.1169,   3.1440],
         [  0.1970,   3.0549,   0.4953,  ...,  -3.7254,  -1.1662,  -2.3692]],

        [[ -1.3138,  -3.1416,  -1.2783,  ...,  -9.0740,   0.1233,  -9.4468],
         [ -0.1919,  -9.3639,  -0.4916,  ...,  -2.5112,   1.1731,  -3.8718]],

        ...,

        [[ -0.8423,  -3.1416,  -0.8150,  ...,  -7.3862,  -0.1728,  -7.2680],
         [ -0.3699,  -6.8518,  -0.5396,  ...,  -4.9153,   0.5758,  -5.7780]],

        [[  0.3320,  -3.1416,   0.3201,  ...,  -1.4442,   0.1220,  -1.5735],
         [  0.1948,  -1.8128,   0.2532,  ...,  -1.9584,  -0.1439,  -1.7042]],

        [[ -1.9077,  -3.

In [17]:
KNet_Pipeline = Pipeline_KF("provaaaa", "KNet", "KalmanNet", data_name)
KNet_Pipeline.setssModel(sys_model)
KNet_model = KalmanNetNN_NewArch()
KNet_model.Build(sys_model_KNet)
KNet_Pipeline.model = torch.load(path_to_project + "KNet/Pendulum/NewArch/model_KalmanNet_pendulum(intermediate bound).pt", map_location=dev)
KNet_Pipeline.pendulum_data_flag = 1
KNet_Pipeline.matrix_data_flag = 0
KNet_Pipeline.fix_H_flag = 1
KNet_Pipeline.modelFileName = path_to_project + "KNet/Pendulum/NewArch/model_KalmanNet_pendulum(intermediate bound).pt"

In [9]:
KNet_Pipeline.model.state_dict()

OrderedDict([('GRU_Q.weight_ih_l0',
              tensor([[-0.4724,  0.1290,  0.4314, -0.4028, -0.1589, -0.1970,  0.0475,  0.1803,
                       -0.4618, -0.1400],
                      [-0.4682, -0.0216,  0.4736, -0.0841,  0.4481, -0.1888, -0.3175,  0.1727,
                        0.0114,  0.1236],
                      [-0.1162,  0.2373,  0.0436, -0.1665,  0.2506,  0.4099, -0.1328, -0.0537,
                        0.1288,  0.1468],
                      [ 0.1771,  0.2693, -0.2736, -0.0343,  0.1101,  0.1583,  0.4556,  0.3338,
                       -0.4282,  0.3601],
                      [ 0.4637, -0.2395, -0.3975,  0.2858,  0.0845,  0.1997, -0.3866, -0.1194,
                       -0.3496,  0.3922],
                      [-0.0590, -0.4512,  0.1705, -0.2504,  0.2093,  0.1657, -0.4321, -0.4621,
                       -0.0319,  0.1300],
                      [ 0.2138, -0.3269,  0.3589,  0.2524, -0.0246, -0.3975,  0.4404,  0.0010,
                       -0.4293,  0.0873],
     

In [ ]:
KNet_Pipeline.NNTest(N_T, test_input, test_target, None, model_AE_conv_trained, 0)

In [7]:
encoder_mod = Encoder(encoded_space_dim=1)
encoder_mod.load_state_dict(torch.load(r"C:\Users\damis\Google Drive\ETH\Master thesis\KalmanNet_VO\saved_models\training_new_big_dataset\only_conv_encoder.pt"))
test_encoder(encoder_mod, test_input, test_target)

output shape: torch.Size([4000, 1])
MSE TEST ENCODER-ONLY LINEAR-SCALE:161.4914093017578 
MSE TEST ENCODER-ONLY LOG-SCALE: 22.081497192382812 dB 


In [ ]:
o=test_input[0,...]
ae_inp = o[0,...].reshape(1, 1, 24, 24) / 255
KNet_Pipeline.model.eval()
out = KNet_Pipeline.model(model_AE_conv_trained(ae_inp), KNet_Pipeline.fix_H_flag)

In [ ]:
rrr = test_input.reshape(4000,24,24)
rrr[0]

tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,  

In [ ]:
test_target_resh = test_target.reshape(4000,2)
print(test_target_resh[0,:])
print(test_target[0,:,0])

tensor([ 0.3192, -3.1416])
tensor([0.3192, 0.1884])


In [ ]:
x_out_test = torch.empty(KNet_Pipeline.ssModel.m, KNet_Pipeline.ssModel.T_test)
for t in range(0, KNet_Pipeline.ssModel.T_test):
    AE_input = o[t, :, :].reshape(1, 1, 24, 24) / 255
    y_test_decoaded_t = model_AE_conv_trained(AE_input)
    print(y_test_decoaded_t.shape)
    x_out_test[:, t] = KNet_Pipeline.model(y_test_decoaded_t, KNet_Pipeline.fix_H_flag)

torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])


In [ ]:
x_out_test[1,:].shape

torch.Size([40])

In [94]:
x = torch.tensor([1,2,3,4])
x = torch.unsqueeze(x, 2)
print(x.shape)
print(x)

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [ ]:
test_target.shape

torch.Size([100, 2, 40])